# Getting Started Tutorial

To install Evidently using the pip package manager, run:

```$ pip install evidently```


If you want to see reports inside a Jupyter notebook, you need to also install the Jupyter nbextension. After installing evidently, run the two following commands in the terminal from the Evidently directory.

To install jupyter nbextension, run:

```$ jupyter nbextension install --sys-prefix --symlink --overwrite --py evidently```

To enable it, run:

```$ jupyter nbextension enable evidently --py --sys-prefix```

That's it!

In [1]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

  Cloning https://github.com/evidentlyai/evidently.git to c:\users\alexb\appdata\local\temp\pip-req-build-ii7ubmun
  Resolved https://github.com/evidentlyai/evidently.git to commit e4f564ce03c7d586b333d243e8282b7ace56fe24
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 15.6/15.6 MB 8.0 MB/s eta 0:00:00
  Using cached statsmodels-0.14.0-cp38-cp38-win_amd64.whl (9.4 MB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ---------------------------------------- 66.3/66.3 kB 3.7 MB/s eta 0:00:00
  Using cached watchdog-3.0.0-py3-none-win_amd64.whl (82 kB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
     -------------------------------------- 239.8/239.8 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 5.1 MB/s eta 0:00:00
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
     -------

  Running command git clone --filter=blob:none --quiet https://github.com/evidentlyai/evidently.git 'C:\Users\alexb\AppData\Local\Temp\pip-req-build-ii7ubmun'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.38.0 requires crcmod<2.0,>=1.7, which is not installed.
apache-beam 2.38.0 requires fastavro<2,>=0.23.6, which is not installed.
apache-beam 2.38.0 requires grpcio<2,>=1.29.0, which is not installed.
apache-beam 2.38.0 requires hdfs<3.0.0,>=2.1.0, which is not installed.
apache-beam 2.38.0 requires httplib2<0.20.0,>=0.8, which is not installed.
apache-beam 2.38.0 requires oauth2client<5,>=2.0.1, which is not installed.
apache-beam 2.38.0 requires proto-plus<2,>=1.7.1, which is not installed.
apache-beam 2.38.0 requires protobuf<4,>=3.12.2, which is not installed.
apache-beam 2.38.0 requires pydot<2,>=1.2.0, which is not installed.
apache-beam 2

In [2]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Load Data

In [4]:
data = fetch_california_housing(as_frame=True)
housing_data = data.frame

In [5]:
housing_data.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing_data['prediction'] = housing_data['target'].values + np.random.normal(0, 5, housing_data.shape[0])

In [6]:
reference = housing_data.sample(n=5000, replace=False)
current = housing_data.sample(n=5000, replace=False)

## Report

In [7]:
report = Report(metrics=[
    DataDriftPreset(), 
])

report.run(reference_data=reference, current_data=current)
report

In [8]:
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AveRooms'),
    ColumnQuantileMetric(column_name='AveRooms', quantile=0.25),
    ColumnDriftMetric(column_name='AveRooms'),
    
])

report.run(reference_data=reference, current_data=current)
report

In [ ]:
report = Report(metrics=[
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile':0.25}, columns=['AveRooms', 'AveBedrms']),
])

report.run(reference_data=reference, current_data=current)
report

In [ ]:
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AveRooms'),
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile':0.25}, columns='num'),
    DataDriftPreset()
])

report.run(reference_data=reference, current_data=current)
report

In [ ]:
report.save_html('report.html')

In [ ]:
report.as_dict()

In [ ]:
report.json()

In [ ]:
#report.save_json('report.json')

## Test Suite 

In [ ]:
tests = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
])

tests.run(reference_data=reference, current_data=current)
tests

In [ ]:
suite = TestSuite(tests=[
    NoTargetPerformanceTestPreset(),
])

suite.run(reference_data=reference, current_data=current)
suite

In [ ]:
suite = TestSuite(tests=[
    TestColumnDrift('Population'),
    TestMeanInNSigmas('HouseAge'),
    NoTargetPerformanceTestPreset(columns=['AveRooms', 'AveBedrms', 'AveOccup'])
])

suite.run(reference_data=reference, current_data=current)
suite

In [ ]:
suite = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
    TestColumnDrift('Population'),
    TestShareOfOutRangeValues('Population'),
    DataStabilityTestPreset(),
    RegressionTestPreset()
    
])

suite.run(reference_data=reference, current_data=current)
suite

In [ ]:
suite.as_dict()

In [ ]:
suite.json()

In [ ]:
suite.save_html('test_suite.html')

In [ ]:
#suite.save_json('test_suite.json')

# Support Evidently
Did you find the example useful? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently